In [20]:
import sys
import pandas as pd
import numpy as np
from enum import Enum

from aocutils.get_aoc_inputs import get_aoc_inputs

In [60]:
if 'inputs' not in locals():
    inputs = get_aoc_inputs(3)

class LineType(Enum):
    VERTICAL=1
    HORIZONTAL=2

class WireSegment:
    def __init__(self, x, y, instruction):
        direction = instruction[0]
        self.length = int(instruction[1:])
        self.startx = x
        self.starty = y

        if( direction == "U" ):
            self.direction = LineType.VERTICAL
            self.endx = x
            self.endy = y + self.length
            self.x1 = x
            self.x2 = x
            self.y1 = y
            self.y2 = self.endy
        elif( direction == "D" ):
            self.direction = LineType.VERTICAL
            self.endx = x
            self.endy = y - self.length
            self.x1 = x
            self.x2 = x
            self.y1 = self.endy
            self.y2 = y 
        elif( direction == "L" ):
            self.direction = LineType.HORIZONTAL
            self.endx = x - self.length
            self.endy = y
            self.x1 = self.endx
            self.x2 = x
            self.y1 = y 
            self.y2 = y 
        elif( direction == "R" ):
            self.direction = LineType.HORIZONTAL
            self.endx = x + self.length
            self.endy = y
            self.x1 = x 
            self.x2 = self.endx
            self.y1 = y 
            self.y2 = y 

    def __str__(self):
        if( self.direction == LineType.VERTICAL ):
            direction = "V"
        else: 
            direction = "H"

        return "%s%d(%d,%d) -> (%d,%d)" % (direction,self.length,self.startx,self.starty,self.endx,self.endy)

class Wire:
    def __init__(self, instructions):
        x=0
        y=0
        segments = []
        for instruction in instructions:
            segment = WireSegment(x,y,instruction)
#             print(segment)
            segments.append(segment)
            x = segment.endx
            y = segment.endy
        
        self.segments = segments

            
def get_wires(wire_diagrams):
    wires = []
    for wire_diagram in wire_diagrams:
        if( isinstance(wire_diagram,str) ):
            wire_diagram = wire_diagram.split(",")
        
        wires.append(Wire(wire_diagram))

    return wires

"""
H(0,0) -> (8,0)
V(8,0) -> (8,5)
H(8,5) -> (3,5)* x1=3 x2=8 | y1=y2=5
V(3,5) -> (3,2)
V(0,0) -> (0,7)
H(0,7) -> (6,7)
V(6,7) -> (6,3)* x1=x2=6   | y1=3 y2=7
H(6,3) -> (2,3)
"""


def closest_crossover(wires):
    assert len(wires) == 2, "Only support 2 wires right now"
    shortest_distance = sys.maxsize
    for wire1 in wires[0].segments:
        for wire2 in wires[1].segments:
            if wire1.direction == LineType.VERTICAL and wire2.direction == LineType.HORIZONTAL:
                if (wire1.x1 >= wire2.x1 and wire1.x1 <= wire2.x2) and (wire2.y1 >= wire1.y1 and wire2.y1 <= wire1.y2 ):
                    distance = abs(wire1.x1) + abs(wire2.y1)
                    if distance > 0 and distance < shortest_distance:
#                         print("Intersection between [%s]x[%s] @ (%d,%d)" % (wire1, wire2, wire1.x1, wire2.y1))
                        shortest_distance = distance
            elif wire1.direction == LineType.HORIZONTAL and wire2.direction == LineType.VERTICAL:
                if (wire2.x1 >= wire1.x1 and wire2.x1 <= wire1.x2) and (wire1.y1 >= wire2.y1 and wire1.y1 <= wire2.y2 ):
                    distance = abs(wire2.x1) + abs(wire1.y1)
                    if distance > 0 and distance < shortest_distance:
#                         print("Intersection between [%s]x[%s] @ (%d,%d)" % (wire1, wire2, wire2.x1, wire1.y1))
                        shortest_distance = distance

    
    return shortest_distance
        

test_inputs_1 = [
    {"test_in":["R8,U5,L5,D3","U7,R6,D4,L4"], "test_out":6},
    {"test_in":["R75,D30,R83,U83,L12,D49,R71,U7,L72","U62,R66,U55,R34,D71,R55,D58,R83"], "test_out":159},
    {"test_in":["R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51","U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"], "test_out":135},
]
    
def test_part1():
    for test_input in test_inputs_1:
        wires = get_wires(test_input["test_in"])
        shortest_distance = closest_crossover(wires)
        assert shortest_distance == test_input["test_out"], "Shortest distance was %d but should be %d" % (shortest_distance, test_input["test_out"])
        
    print("Test Part 1 Successful")

        
def day1():
    wires = get_wires(inputs)
    shortest_distance = closest_crossover(wires)
    print("Shortest distance: %d" % shortest_distance)
        


test_part1()
day1()

# ------------------------------------------------------------------------------------------

test_inputs_2 = [
    {"test_in":["R8,U5,L5,D3","U7,R6,D4,L4"], "test_out":30},
    {"test_in":["R75,D30,R83,U83,L12,D49,R71,U7,L72","U62,R66,U55,R34,D71,R55,D58,R83"], "test_out":610},
    {"test_in":["R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51","U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"], "test_out":410},
]

def shortest_walk(wires):
    assert len(wires) == 2, "Only support 2 wires right now"
    shortest_distance = sys.maxsize

    wire1_walk=0
    for wire1 in wires[0].segments:

        wire2_walk=0
        for wire2 in wires[1].segments:
#             print("> Processing %s x %s" % (wire1, wire2))
            
            if wire1.direction == LineType.VERTICAL and wire2.direction == LineType.HORIZONTAL:
                if (wire1.x1 >= wire2.x1 and wire1.x1 <= wire2.x2) and (wire2.y1 >= wire1.y1 and wire2.y1 <= wire1.y2 ):
#                     print(">   Intersection between [%s]x[%s] @ (%d,%d)" % (wire1, wire2, wire1.x1, wire2.y1))
#                     print(">   Wire1 So Far %d, this segment wire1 %d, Wire2 So Far %d, this segment wire2 %d" % (wire1_walk,abs(wire1.starty-wire2.y1), wire2_walk, abs(wire2.startx-wire1.x1)))
                    total_walk = wire1_walk + abs(wire1.starty-wire2.y1) + wire2_walk + abs(wire2.startx-wire1.x1)
                    if total_walk > 0 and total_walk < shortest_distance:
#                         print(">   New shortest walk! %d" % total_walk)
                        shortest_distance = total_walk
    
            elif wire1.direction == LineType.HORIZONTAL and wire2.direction == LineType.VERTICAL:
                if (wire2.x1 >= wire1.x1 and wire2.x1 <= wire1.x2) and (wire1.y1 >= wire2.y1 and wire1.y1 <= wire2.y2 ):
#                     print(">   Intersection between [%s]x[%s] @ (%d,%d)" % (wire1, wire2, wire2.x1, wire1.y1))
#                     print(">   Wire1 So Far %d, this segment wire1 %d, Wire2 So Far %d, this segment wire2 %d" % (wire1_walk,abs(wire1.startx-wire2.x1), wire2_walk, abs(wire2.starty-wire1.y1)))
                    total_walk = wire1_walk + abs(wire1.startx-wire2.x1) + wire2_walk + abs(wire2.starty-wire1.y1)
                    distance = abs(wire2.x1) + abs(wire1.y1)
                    if total_walk > 0 and total_walk < shortest_distance:
#                         print(">   New shortest walk! %d" % total_walk)
                        shortest_distance = total_walk
                        
#             else:
#                 print(">   No Intersection")
    
            wire2_walk += wire2.length
#             print("Wire2 now: %d" % wire2_walk)
        wire1_walk += wire1.length
#         print("Wire1 now: %d" % wire1_walk)


    
    return shortest_distance


    
def test_part2():
    for test_input in test_inputs_2:
        wires = get_wires(test_input["test_in"])
        shortest_distance = shortest_walk(wires)
        assert shortest_distance == test_input["test_out"], "Shortest walk was %d but should be %d" % (shortest_distance, test_input["test_out"])
        
    print("Test Part 2 Successful")

        
def day2():
    wires = get_wires(inputs)
    shortest_distance = shortest_walk(wires)
    print("Shortest walk: %d" % shortest_distance)

            
test_part2()
day2()



Test Part 1 Successful
Shortest distance: 731
Test Part 2 Successful
Shortest walk: 5672
